# Run Analysis & Data Quality Checks

This notebook validates experimental runs according to the **008-fix-zero-tokens** data model.

## Key Changes (v2.0.0):
- Token metrics (`TOK_IN`, `TOK_OUT`, `API_CALLS`, `CACHED_TOKENS`) are **only** at run-level in `aggregate_metrics`
- Steps array contains **only** timing and status data (no tokens)
- Reconciliation status tracked in `usage_api_reconciliation` section

## What We Check:
1. **Run completeness**: All expected files present
2. **Data model compliance**: No token fields in steps array
3. **Reconciliation status**: Verification progress tracking
4. **Token accuracy**: Run-level totals validation
5. **Step integrity**: Timing and status consistency

In [ ]:
# Import required libraries
import json
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np

# Configuration
RUNS_DIR = '../runs/'  # Relative path from analysis directory

print("📊 Run Analysis Notebook")
print("=" * 60)
print(f"Runs directory: {RUNS_DIR}")
print(f"Analysis started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 60)

## 1. Load and Parse All Runs

Scan the runs directory and load all `metrics.json` files.

In [ ]:
def load_all_runs(runs_directory=RUNS_DIR):
    """Load all metrics.json files from the runs directory."""
    runs_data = []
    missing_metrics = []
    parse_errors = []
    
    runs_path = Path(runs_directory)
    if not runs_path.exists():
        print(f"⚠️  Runs directory not found: {runs_directory}")
        return [], []
    
    # Iterate through framework directories
    for framework_dir in runs_path.iterdir():
        if not framework_dir.is_dir() or framework_dir.name.startswith('.'):
            continue
            
        framework_name = framework_dir.name
        
        # Iterate through run directories
        for run_dir in framework_dir.iterdir():
            if not run_dir.is_dir() or run_dir.name.startswith('.'):
                continue
                
            run_id = run_dir.name
            metrics_file = run_dir / 'metrics.json'
            
            if not metrics_file.exists():
                missing_metrics.append({
                    'framework': framework_name,
                    'run_id': run_id,
                    'issue': 'metrics.json not found'
                })
                continue
            
            try:
                with open(metrics_file, 'r') as f:
                    metrics = json.load(f)
                
                runs_data.append({
                    'framework': framework_name,
                    'run_id': run_id,
                    'metrics_file': str(metrics_file),
                    'data': metrics
                })
            except json.JSONDecodeError as e:
                parse_errors.append({
                    'framework': framework_name,
                    'run_id': run_id,
                    'issue': f'JSON parse error: {str(e)}'
                })
            except Exception as e:
                parse_errors.append({
                    'framework': framework_name,
                    'run_id': run_id,
                    'issue': f'Unexpected error: {str(e)}'
                })
    
    print(f"✓ Found {len(runs_data)} valid runs")
    if missing_metrics:
        print(f"⚠️  {len(missing_metrics)} runs missing metrics.json")
    if parse_errors:
        print(f"❌ {len(parse_errors)} runs with parse errors")
    
    return runs_data, missing_metrics + parse_errors

# Load all runs
runs_data, issues = load_all_runs()
print(f"\n📊 Total runs loaded: {len(runs_data)}")

## 2. Data Model Compliance Check

Verify that runs follow the v2.0.0 data model (no token fields in steps array).

In [ ]:
def check_data_model_compliance(runs_data):
    """Check that runs follow v2.0.0 data model (no tokens in steps)."""
    violations = []
    
    # Token fields that should NOT be in steps
    forbidden_fields = ['tokens_in', 'tokens_out', 'api_calls', 'cached_tokens']
    
    for run in runs_data:
        framework = run['framework']
        run_id = run['run_id']
        metrics = run['data']
        
        # Check steps array
        steps = metrics.get('steps', [])
        for step_idx, step in enumerate(steps):
            for field in forbidden_fields:
                if field in step:
                    violations.append({
                        'framework': framework,
                        'run_id': run_id,
                        'step': step.get('step', step_idx + 1),
                        'violation': f'Forbidden field "{field}" in steps array',
                        'value': step[field]
                    })
    
    if violations:
        print(f"❌ Found {len(violations)} data model violations")
        violations_df = pd.DataFrame(violations)
        return violations_df
    else:
        print("✅ All runs comply with v2.0.0 data model")
        return pd.DataFrame()

# Check compliance
violations_df = check_data_model_compliance(runs_data)
if not violations_df.empty:
    print("\nViolations by framework:")
    print(violations_df.groupby('framework').size())
    display(violations_df.head(10))

## 3. Reconciliation Status Overview

Check the reconciliation status for all runs.

In [ ]:
def analyze_reconciliation_status(runs_data):
    """Analyze reconciliation status across all runs."""
    status_data = []
    
    for run in runs_data:
        framework = run['framework']
        run_id = run['run_id']
        metrics = run['data']
        
        reconciliation = metrics.get('usage_api_reconciliation', {})
        status = reconciliation.get('verification_status', 'unknown')
        attempts = reconciliation.get('attempts', [])
        verified_at = reconciliation.get('verified_at')
        
        # Get latest attempt details
        latest_attempt = attempts[-1] if attempts else {}
        
        # Handle verified_at - could be timestamp, string, or None
        verified_at_str = None
        if verified_at:
            if isinstance(verified_at, str):
                # Already a string, use as-is
                verified_at_str = verified_at
            elif isinstance(verified_at, (int, float)):
                # It's a timestamp, convert it
                try:
                    verified_at_str = datetime.fromtimestamp(verified_at).strftime('%Y-%m-%d %H:%M:%S')
                except (ValueError, OSError):
                    # Invalid timestamp
                    verified_at_str = f"Invalid timestamp: {verified_at}"
        
        status_data.append({
            'framework': framework,
            'run_id': run_id,
            'status': status,
            'attempt_count': len(attempts),
            'verified_at': verified_at_str,
            'latest_tokens_in': latest_attempt.get('total_tokens_in', 0),
            'latest_tokens_out': latest_attempt.get('total_tokens_out', 0),
            'latest_api_calls': latest_attempt.get('total_api_calls', 0)
        })
    
    df = pd.DataFrame(status_data)
    
    print("📊 Reconciliation Status Summary")
    print("=" * 60)
    if not df.empty:
        print(df['status'].value_counts())
        print(f"\nTotal runs: {len(df)}")
        print(f"Verified: {len(df[df['status'] == 'verified'])}")
        print(f"Pending: {len(df[df['status'] == 'pending'])}")
        print(f"Failed: {len(df[df['status'] == 'failed'])}")
    else:
        print("No runs found")
    
    return df

# Analyze reconciliation
reconciliation_df = analyze_reconciliation_status(runs_data)
if not reconciliation_df.empty:
    display(reconciliation_df.head(10))


## 4. Run-Level Token Analysis

Validate run-level token metrics from `aggregate_metrics`.

In [ ]:
def analyze_token_metrics(runs_data):
    """Analyze run-level token metrics from aggregate_metrics."""
    token_data = []
    issues = []
    
    for run in runs_data:
        framework = run['framework']
        run_id = run['run_id']
        metrics = run['data']
        
        agg = metrics.get('aggregate_metrics', {})
        reconciliation = metrics.get('usage_api_reconciliation', {})
        
        tok_in = agg.get('TOK_IN', 0)
        tok_out = agg.get('TOK_OUT', 0)
        api_calls = agg.get('API_CALLS', 0)
        cached = agg.get('CACHED_TOKENS', 0)
        cost = agg.get('COST_USD', 0)
        status = reconciliation.get('verification_status', 'unknown')
        
        # Check for zero tokens in verified runs
        if status == 'verified' and (tok_in == 0 or tok_out == 0):
            issues.append({
                'framework': framework,
                'run_id': run_id,
                'issue': 'Zero tokens in verified run',
                'tok_in': tok_in,
                'tok_out': tok_out
            })
        
        token_data.append({
            'framework': framework,
            'run_id': run_id,
            'status': status,
            'TOK_IN': tok_in,
            'TOK_OUT': tok_out,
            'API_CALLS': api_calls,
            'CACHED_TOKENS': cached,
            'COST_USD': cost,
            'total_tokens': tok_in + tok_out
        })
    
    df = pd.DataFrame(token_data)
    
    print("📊 Token Metrics Summary")
    print("=" * 60)
    print(f"Total runs: {len(df)}")
    
    if not df.empty:
        print(f"\nToken statistics (all runs):")
        print(df[['TOK_IN', 'TOK_OUT', 'total_tokens', 'COST_USD']].describe())
        print(f"\nRuns with zero input tokens: {len(df[df['TOK_IN'] == 0])}")
        print(f"Runs with zero output tokens: {len(df[df['TOK_OUT'] == 0])}")
    
    if issues:
        print(f"\n⚠️  {len(issues)} verified runs with zero tokens!")
        issues_df = pd.DataFrame(issues)
        return df, issues_df
    
    return df, pd.DataFrame()

# Analyze tokens
token_df, token_issues_df = analyze_token_metrics(runs_data)
if not token_df.empty:
    display(token_df.head(10))


## 5. Step Integrity Check

Validate step-level data (timing, status, counts).

In [ ]:
def check_step_integrity(runs_data):
    """Check step-level data for inconsistencies."""
    step_issues = []
    step_summary = []
    
    for run in runs_data:
        framework = run['framework']
        run_id = run['run_id']
        metrics = run['data']
        
        steps = metrics.get('steps', [])
        
        for step in steps:
            step_num = step.get('step', -1)
            duration = step.get('duration_seconds', 0)
            start_ts = step.get('start_timestamp', 0)
            end_ts = step.get('end_timestamp', 0)
            success = step.get('success', None)
            
            # Check for missing required fields
            if step_num < 0:
                step_issues.append({
                    'framework': framework,
                    'run_id': run_id,
                    'step': step_num,
                    'issue': 'Missing step number'
                })
            
            # Check for negative or zero duration
            if duration <= 0:
                step_issues.append({
                    'framework': framework,
                    'run_id': run_id,
                    'step': step_num,
                    'issue': f'Invalid duration: {duration}'
                })
            
            # Check timestamp consistency
            if start_ts > 0 and end_ts > 0:
                computed_duration = end_ts - start_ts
                if abs(computed_duration - duration) > 1:  # Allow 1 second tolerance
                    step_issues.append({
                        'framework': framework,
                        'run_id': run_id,
                        'step': step_num,
                        'issue': f'Duration mismatch: stored={duration}, computed={computed_duration}'
                    })
            
            # Check success field
            if success is None:
                step_issues.append({
                    'framework': framework,
                    'run_id': run_id,
                    'step': step_num,
                    'issue': 'Missing success status'
                })
            
            step_summary.append({
                'framework': framework,
                'run_id': run_id,
                'step': step_num,
                'duration': duration,
                'success': success,
                'hitl_count': step.get('hitl_count', 0),
                'retry_count': step.get('retry_count', 0)
            })
    
    summary_df = pd.DataFrame(step_summary)
    
    print("📊 Step Integrity Summary")
    print("=" * 60)
    print(f"Total steps analyzed: {len(summary_df)}")
    if not summary_df.empty:
        print(f"Successful steps: {len(summary_df[summary_df['success'] == True])}")
        print(f"Failed steps: {len(summary_df[summary_df['success'] == False])}")
        print(f"\nDuration statistics:")
        print(summary_df['duration'].describe())
    
    if step_issues:
        print(f"\n⚠️  {len(step_issues)} step integrity issues found!")
        issues_df = pd.DataFrame(step_issues)
        return summary_df, issues_df
    else:
        print("\n✅ All steps have valid data")
        return summary_df, pd.DataFrame()

# Check step integrity
steps_df, step_issues_df = check_step_integrity(runs_data)
if not steps_df.empty:
    print(f"\nSteps per run statistics:")
    print(steps_df.groupby(['framework', 'run_id']).size().describe())


## 6. Framework Comparison

Compare metrics across different frameworks.

In [ ]:
# Framework comparison
print("📊 Framework Comparison")
print("=" * 60)

if not token_df.empty:
    # Count runs per framework
    print("\n🔢 Run counts:")
    print(token_df['framework'].value_counts())
    
    # Average metrics by framework
    print("\n📈 Average metrics by framework:")
    framework_summary = token_df.groupby('framework')[['TOK_IN', 'TOK_OUT', 'total_tokens', 'API_CALLS', 'COST_USD']].agg(['mean', 'median', 'std'])
    print(framework_summary)
    
    # Reconciliation status by framework
    if not reconciliation_df.empty:
        print("\n✅ Reconciliation status by framework:")
        status_by_framework = reconciliation_df.groupby(['framework', 'status']).size().unstack(fill_value=0)
        print(status_by_framework)
else:
    print("No token data available for comparison")


## 7. Export Issues to CSV

Save all identified issues for further investigation.

In [ ]:
# Export issues to CSV
export_count = 0

if issues:
    issues_df = pd.DataFrame(issues)
    issues_df.to_csv('issues_missing_metrics.csv', index=False)
    print(f"✓ Exported {len(issues_df)} missing metrics issues to issues_missing_metrics.csv")
    export_count += 1

if not violations_df.empty:
    violations_df.to_csv('issues_data_model_violations.csv', index=False)
    print(f"✓ Exported {len(violations_df)} data model violations to issues_data_model_violations.csv")
    export_count += 1

if not token_issues_df.empty:
    token_issues_df.to_csv('issues_zero_tokens_verified.csv', index=False)
    print(f"✓ Exported {len(token_issues_df)} zero token issues to issues_zero_tokens_verified.csv")
    export_count += 1

if not step_issues_df.empty:
    step_issues_df.to_csv('issues_step_integrity.csv', index=False)
    print(f"✓ Exported {len(step_issues_df)} step integrity issues to issues_step_integrity.csv")
    export_count += 1

# Export pending reconciliations
if not reconciliation_df.empty:
    pending_df = reconciliation_df[reconciliation_df['status'] == 'pending']
    if not pending_df.empty:
        pending_df.to_csv('runs_pending_reconciliation.csv', index=False)
        print(f"✓ Exported {len(pending_df)} pending reconciliations to runs_pending_reconciliation.csv")
        export_count += 1

if export_count > 0:
    print(f"\n✅ Export complete! {export_count} file(s) created.")
else:
    print("\n✅ No issues to export - all data is clean!")


## 8. Visualizations

Generate charts for token usage and reconciliation status.

In [ ]:
# Install visualization libraries if needed
%pip install matplotlib seaborn -q

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Create visualizations directory
os.makedirs('visualizations', exist_ok=True)

if not token_df.empty and len(token_df) > 0:
    # 1. Token distribution by framework
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Total tokens by framework
    token_df.groupby('framework')['total_tokens'].mean().plot(kind='bar', ax=axes[0], color='steelblue')
    axes[0].set_title('Average Total Tokens by Framework')
    axes[0].set_ylabel('Total Tokens')
    axes[0].set_xlabel('Framework')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Cost by framework
    token_df.groupby('framework')['COST_USD'].mean().plot(kind='bar', ax=axes[1], color='coral')
    axes[1].set_title('Average Cost (USD) by Framework')
    axes[1].set_ylabel('Cost (USD)')
    axes[1].set_xlabel('Framework')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('visualizations/token_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Saved visualization to visualizations/token_analysis.png")
else:
    print("⚠️  No token data available for visualization")


In [ ]:
# 2. Reconciliation status distribution
if not reconciliation_df.empty and len(reconciliation_df) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    status_counts = reconciliation_df['status'].value_counts()
    colors = {'verified': 'green', 'pending': 'orange', 'failed': 'red', 'data_not_available': 'gray'}
    bar_colors = [colors.get(status, 'blue') for status in status_counts.index]
    
    status_counts.plot(kind='bar', ax=ax, color=bar_colors)
    ax.set_title('Reconciliation Status Distribution', fontsize=14, fontweight='bold')
    ax.set_ylabel('Number of Runs')
    ax.set_xlabel('Status')
    ax.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for i, v in enumerate(status_counts):
        ax.text(i, v + 0.1, str(v), ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('visualizations/reconciliation_status.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Saved visualization to visualizations/reconciliation_status.png")
else:
    print("⚠️  No reconciliation data available for visualization")


In [ ]:
# 3. Token distribution histogram
if not token_df.empty and len(token_df[token_df['TOK_IN'] > 0]) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Input tokens
    token_df[token_df['TOK_IN'] > 0]['TOK_IN'].hist(bins=30, ax=axes[0, 0], color='skyblue', edgecolor='black')
    axes[0, 0].set_title('Distribution of Input Tokens (excluding zeros)')
    axes[0, 0].set_xlabel('Input Tokens')
    axes[0, 0].set_ylabel('Frequency')
    
    # Output tokens
    token_df[token_df['TOK_OUT'] > 0]['TOK_OUT'].hist(bins=30, ax=axes[0, 1], color='lightcoral', edgecolor='black')
    axes[0, 1].set_title('Distribution of Output Tokens (excluding zeros)')
    axes[0, 1].set_xlabel('Output Tokens')
    axes[0, 1].set_ylabel('Frequency')
    
    # API calls
    token_df[token_df['API_CALLS'] > 0]['API_CALLS'].hist(bins=30, ax=axes[1, 0], color='lightgreen', edgecolor='black')
    axes[1, 0].set_title('Distribution of API Calls (excluding zeros)')
    axes[1, 0].set_xlabel('API Calls')
    axes[1, 0].set_ylabel('Frequency')
    
    # Cost
    token_df[token_df['COST_USD'] > 0]['COST_USD'].hist(bins=30, ax=axes[1, 1], color='gold', edgecolor='black')
    axes[1, 1].set_title('Distribution of Cost in USD (excluding zeros)')
    axes[1, 1].set_xlabel('Cost (USD)')
    axes[1, 1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig('visualizations/token_distributions.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("✓ Saved visualization to visualizations/token_distributions.png")
else:
    print("⚠️  No token data available for distribution histograms")

## 9. Summary Report

Generate a comprehensive summary of the analysis.

In [ ]:
print("=" * 80)
print("EXPERIMENT RUN ANALYSIS SUMMARY REPORT")
print("=" * 80)
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Runs Directory: {RUNS_DIR}")
print()

print("📊 DATA QUALITY OVERVIEW")
print("-" * 80)
print(f"Total runs analyzed: {len(runs_data)}")
print(f"Runs with missing metrics: {len([i for i in issues if 'metrics.json not found' in i.get('issue', '')])}")
print(f"Runs with parse errors: {len([i for i in issues if 'parse error' in i.get('issue', '').lower()])}")
print()

print("✅ DATA MODEL COMPLIANCE (v2.0.0)")
print("-" * 80)
if violations_df.empty:
    print("✓ All runs comply with v2.0.0 data model")
    print("  - No token fields in steps array")
    print("  - Token metrics stored only in aggregate_metrics")
else:
    print(f"❌ {len(violations_df)} data model violations found")
    print(f"  - Runs with violations: {violations_df['run_id'].nunique()}")
    print(f"  - See: issues_data_model_violations.csv")
print()

print("🔄 RECONCILIATION STATUS")
print("-" * 80)
if not reconciliation_df.empty:
    for status, count in reconciliation_df['status'].value_counts().items():
        percentage = (count / len(reconciliation_df)) * 100
        print(f"  {status}: {count} runs ({percentage:.1f}%)")
else:
    print("  No reconciliation data available")
print()

print("💰 TOKEN & COST METRICS")
print("-" * 80)
if not token_df.empty:
    verified_runs = token_df[token_df['status'] == 'verified']
    if not verified_runs.empty:
        print(f"Verified runs: {len(verified_runs)}")
        print(f"  Average input tokens: {verified_runs['TOK_IN'].mean():.0f}")
        print(f"  Average output tokens: {verified_runs['TOK_OUT'].mean():.0f}")
        print(f"  Average API calls: {verified_runs['API_CALLS'].mean():.1f}")
        print(f"  Average cost: ${verified_runs['COST_USD'].mean():.4f}")
        print(f"  Total cost (all verified): ${verified_runs['COST_USD'].sum():.2f}")
    else:
        print("  No verified runs found")
else:
    print("  No token data available")
print()

if not token_issues_df.empty:
    print(f"⚠️  WARNING: {len(token_issues_df)} verified runs with zero tokens")
    print(f"  - See: issues_zero_tokens_verified.csv")
    print()

print("📝 STEP ANALYSIS")
print("-" * 80)
if not steps_df.empty:
    print(f"Total steps: {len(steps_df)}")
    print(f"Average steps per run: {steps_df.groupby('run_id').size().mean():.1f}")
    print(f"Successful steps: {len(steps_df[steps_df['success'] == True])}")
    print(f"Failed steps: {len(steps_df[steps_df['success'] == False])}")
    print(f"Average step duration: {steps_df['duration'].mean():.2f} seconds")
else:
    print("  No step data found")
print()

if not step_issues_df.empty:
    print(f"⚠️  {len(step_issues_df)} step integrity issues found")
    print(f"  - See: issues_step_integrity.csv")
    print()

print("🎯 FRAMEWORK BREAKDOWN")
print("-" * 80)
if not token_df.empty:
    for framework in token_df['framework'].unique():
        fw_runs = token_df[token_df['framework'] == framework]
        fw_verified = fw_runs[fw_runs['status'] == 'verified']
        print(f"{framework}:")
        print(f"  Total runs: {len(fw_runs)}")
        print(f"  Verified: {len(fw_verified)}")
        if not fw_verified.empty:
            print(f"  Avg tokens: {fw_verified['total_tokens'].mean():.0f}")
            print(f"  Avg cost: ${fw_verified['COST_USD'].mean():.4f}")
        print()
else:
    print("  No framework data available")

print("=" * 80)
print("✅ Analysis complete! Check the 'visualizations/' directory for charts.")
print("=" * 80)
